In [43]:
from docx import Document
import os
import re
import csv
import sys

In [44]:
'''
Author: Eden

Read stopwords from files and return a stopwords set.

-filename = file name

return the set of stopwords
'''
def read_stop_words(filename):
    vector = []
    with open(filename, 'r') as f:
        input = f.read().splitlines()

    [vector.append(input[i]) for i in range(len(input))]   
    return set(vector)

In [45]:
'''
Author: Zoe

Remove the stopwords such as articles, prep and so on from keywords

-keywords = search word

return the string without stopwords
'''

def remove_stopwords(keywords, stop_set):
    word_list = re.split(r'\s',keywords)
    filtered_words = [word for word in word_list if word not in stop_set]
    words = " ".join(filtered_words)
    return words

### indent(para) Example
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Change of control means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Code shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [46]:
'''
Author: Yufei

Get the indent space of the paragraph, used in find_patterns and match function

-para = paragraph

return a num means the indent space or null if no indent
'''
def indent(para):
    if para.paragraph_format.first_line_indent:
        return para.paragraph_format.first_line_indent.pt
    else:
        return None

### bullet_list(para) Example
Change of control means that 

&nbsp;&nbsp;&nbsp;&nbsp;(a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 

&nbsp;&nbsp;&nbsp;&nbsp;(b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.


In [47]:
'''
Author: Eden

Check if the paragraph is bullet ordered list, used in find_patterns and match function

-para = paragraph

return the regex pattern of this paragraph or None
'''
def bullet_list(para):
    pattern = '[A-Z]+\)'
    flag = re.match(pattern, para.text.lstrip()[:10])
              
    if flag:
        return pattern
    
    pattern = '[a-z]+\)'
    flag = re.match(pattern, para.text.lstrip()[:10])
    
    if flag:
        return pattern
    
    pattern = '\d+\)'
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    pattern = '\d+\.[^\d]+'
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    pattern = '[a-z]+\.'
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    pattern = '[A-Z]+\.'
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    
    pattern = '\d+\.\d+[^\d]+'
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    pattern = '\d+\.\d+\.\d+[^\d]+'
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    pattern = '\d+\.\d+\.\d+\.\d+[^\d]+'
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    pattern = '[\(]?([vx]|i{1,3}|i[vx]|[vx]i{1,3})(\.|\))'
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    return None

In [6]:
'''
Testing Function
'''

# Input folder path
directory_path = os.getcwd() + '/input/change_of_control+change_in_control' 
fileName = '1Y0ki1eVSBHxQNmWIfj6Or.docx'
document = Document(directory_path + "/" +fileName)
    
for p in document.paragraphs:
    pattern = bullet_list(p)
    if pattern:
        print pattern
        print p.text

\d+\.[^\d]+
1. Employment.
\d+\.[^\d]+
2. Term.
\d+\.[^\d]+
3. Duties.
\d+\.[^\d]+
4. Compensation, Benefits, Reimbursement and Bonus.
[a-z]+\.
a. Base Salary. In consideration for all services rendered by Executive during the term of this Agreement, Employer shall pay Executive an annual base salary (before all customary and proper payroll deductions) of $200,000, as adjusted from time to time (“Base Salary”). The Board of Directors of the Employer shall review Executive’s salary each year, in a manner consistent with that used for all management employees of the Employer, and in its sole discretion may adjust such salary commensurate with the Executive’s performance under this Agreement.
[a-z]+\.
b. Incentive Compensation. Under the Employer’s Executive Incentive Compensation Plan, Executive shall be eligible to receive an annual bonus based on performance as defined by the Board of Directors. Executive’s annual target bonus will equal 30% of Base Salary. This is the amount payable f

### isListPara(para) Example
1. any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 
2. a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [48]:
'''
Author: Alan

Check if the paragraph is list paragraph rather than normal paragraph, used in find_patterns and match function

-para = paragraph

return True/False
'''
def isListPara(para):
    paraFormat = para.style.name
    if paraFormat == 'List Paragraph':
        return True
    return False

### isNormal(para) Example
any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 

In [49]:
'''
Author: Alan

Check if the paragraph is Normal rather than list paragraph，used in match function

-para = paragraph

return True/False
'''
def isNormal(para):
    paraFormat = para.style.name
    if paraFormat == 'Normal':
        return True
    return False

### isNormalFollwingListPara(paras, curIndex) Example
Change of control means that 
1. any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 
2. a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [50]:
'''
Author: Alan

Check if the paragraph followed by list paragraph, used in find_patterns function

-paras = all the paragraph
-curIndex = the index of current paragraph

return True/False
'''
def isNormalFollowingListPara(paras, curIndex):
    targetFormat = paras[curIndex].style.name
    if curIndex == len(paras) - 1:
        return False
    nextFormat = paras[curIndex+1].style.name
    if nextFormat == 'List Paragraph' and targetFormat == 'Normal':
        return True
    return False

### bold(para) Example
**Change of control** means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

**Code** shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [51]:
'''
Author: Zheng

Check if the bold font in the paragraph, used in match function

-para = paragraph

return True/False
'''
def bold(para):
    paras_toPrint = []
    for run in para.runs:
        if run.bold:
            #print run.text,'bold'
            paras_toPrint.append(run.text)              
    if len(paras_toPrint) == 0:
        return False
    else:
        return True

### is_bold(para, keywords) Example
**Change of control** means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [52]:
'''
Author: Zheng

Check if the bold font and keyword in the paragraph, used in find_patterns function

-para = paragraph
-keywords = search words

return True/False
'''
def is_bold(para, keywords):
    paras_toPrint = []    
    if keywords.lower() in para.text.lower():
        for run in para.runs:
            if run.bold and run.text.lower()==keywords.lower():
                print run.text,'bold'
                paras_toPrint.append(run.text)
    if len(paras_toPrint) == 0:
        return False
    else:
        return True

### is_italic(para, keywords) Example
_Change of control_ means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [53]:
'''
Author: Zheng

Check if the italic font and keywords in the paragraph, used in match function

-para = paragraph

return True/False
'''
def is_italic(para, keywords):
    paras_toPrint = []    
    if keywords.lower() in para.text.lower():
        for run in para.runs:
            if run.italic and run.text.lower()==keywords.lower():
                print run.text,'italic'
                paras_toPrint.append(run.text)
    if len(paras_toPrint) == 0:
        return False
    else:
        return True

### italic(para) Example
_Code_ shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [54]:
'''
Author: Zheng

Check if the italic font in the paragraph, used in match function

-para = paragraph

return True/False
'''
def italic(para):
    paras_toPrint = []
    for run in para.runs:
        if run.italic:
            #print run.text,'italic'
            return True
            paras_toPrint.append(run.text)
    return False

### is_underline(para, keywords) Example
Change of control(Underline) means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [55]:
# Zheng
def is_underline(para, keywords):
    paras_toPrint = []    
    if keywords.lower() in para.text.lower():
        for run in para.runs:
            if run.underline and run.text.lower()==keywords.lower():
                print run.text,'underline'
                paras_toPrint.append(run.text)
    if len(paras_toPrint) == 0:
        return False
    else:
        return True

### underline(para) Example

Code(Underline) shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [56]:
'''
Author: Zheng

Check if the underline in the paragraph, used in match function

-para = paragraph

return True/False
'''
def underline(para):
    paras_toPrint = []
    for run in para.runs:
        if run.underline:
            return True
            #print run.text,'underline'
            paras_toPrint.append(run.text)
    return False

### is_double_quotes(para, keywords) Example:
"Change of control" means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [57]:
'''
Author: Zoe

Check if the keyword is double quoted in the paragraph, used in find_patterns function

-para = paragraph

return True/False
'''
def is_double_quotes(para, keywords, stop_set):
    words = re.split(u'[\"\u201c\u201d]',para.text)
    removed_words = remove_stopwords(keyword, stop_set).split(" ")
    #print words
#     print '\tIS Double Quotes', words
    #print re.match('[a-zA-Z ]*[a-zA-Z]+[a-zA-Z ]*',words[0])
    if len(words) == 1:  # no double quotes
        return False
    # check if all keywords are in words[1], the first phrase in double quotes, and words[0] is not regular phrase which means that words[1] is the start of the paragragh
    if all(word in words[1] for word in removed_words) and len(words[0].rstrip().lstrip()) < 7 and re.match(r'[a-zA-Z ]*[a-zA-Z]+[a-zA-Z ]*',words[0]) == None:
        return True
    return False
#     if not words:
#         return False
#     if para.text.startswith("\"" + words[0] + "\""):   #and words[0].lower() == keyword.lower():
#         return True       
#     else:
#         return False

### double_quotes(para) Example:

"Code" shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [58]:
'''
Author: Zoe

Check if the double quotes in the paragraph, used in match function

-para = paragraph

return True/False
'''
def double_quotes(para):
    """
        Find if paragraph starts with double quotes
        
        return: True or False
    """
    words = re.split(u'[\"\u201c\u201d]',para.text)
    #print words
    if  len(words) == 1:
        return False
    # if words[0] is not regular phrase, we regard  words[1] as the start of the paragragh
    if len(words[0].rstrip().lstrip()) < 7 and re.match(r'[a-zA-Z ]*[a-zA-Z]+[a-zA-Z ]*',words[0]) == None:
        return True       
    else:
        return False

### is_single_quotes(para, keywords) Example:
'Change of control' means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.


In [59]:
'''
Author: Zoe

Check if the keyword is single quoted in the paragraph, used in find_patterns function

-para = paragraph

return True/False
'''
def is_single_quotes(para, keywords, stop_set):
    words = re.split(u'[\"\u2018\u2019]',para.text)
    removed_words = remove_stopwords(keyword, stop_set).split(" ")
    #re.findall(r'[\"\u201C\u201D].*[\"\u201C\u201D]', para.text)
    #print words
#     print '\tIS Double Quotes', words
    if len(words) == 1:    # no single quotes
        return False
        # check if all keywords are in words[1], the first phrase in single quotes, and words[0] is not regular phrase which means that words[1] is the start of the paragragh
    if all(word in words[1] for word in removed_words) and len(words[0].rstrip().lstrip()) < 7 and re.match(r'[a-zA-Z ]*[a-zA-Z]+[a-zA-Z ]*',words[0]) == None:
        return True
    return False

### single_quotes(para) Example:

'Code' shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [60]:
'''
Author: Zoe

Check if single quotes in the paragraph, used in match function

-para = paragraph

return True/False
'''
def single_quotes(para):
    words = re.split(u'[\"\u2018\u2019]',para.text)
    if len(words) == 1:
        return False
    # if words[0] is not regular phrase, we regard words[1] as the start of the paragragh
    if len(words[0].rstrip().lstrip()) < 7 and re.match(r'[a-zA-Z ]*[a-zA-Z]+[a-zA-Z ]*',words[0]) == None:
        return True       
    else:
        return False

### is_upper_case(para, keywords) Example:
CHANGE OF CONTROL shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [61]:
'''
Author: Zoe

Check if the keyword is upper case in the paragraph, used in find_patterns function

-para = paragraph

return True/False
'''
def is_upper_case(para, keywords):
    words = re.findall(r'([A-Z]+?)\s',para.text)
    if not words:
        return False
    if para.text.startswith(words[0]): # and words[0].lower() == keyword.lower():
        return True
    else:
        return False

### upper_case(para) Example:

UPPERCASE shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [62]:
'''
Author: Zoe

Check if upper case word in the paragraph, used in match function

-para = paragraph

return True/False
'''
def upper_case(para):
    words = re.findall(r'([A-Z]+?)\s',para.text)
    if not words:
        return False
    if para.text.startswith(words[0]):
        return True
    else:
        return False

#### Remove_Stopwords Test

In [63]:
'''
Testing Function
'''
directory_path = 'res' 
fileName = 'stop_words.txt'
stop_set = read_stop_words(directory_path + '/' +  fileName)
keyword = "Change of Control"
print remove_stopwords(keyword, stop_set)

Change Control


In [64]:
'''
Author: Eden

Traversing through each of the paragraphs and find the paragraph containing the keywords

-paras = all paragraphs
-keywords = search words

return a list of paragraph index 
'''
def search_2(paras, keyword):
    index = []
    i = 0
    target = remove_stopwords(keyword.lower(), stop_set)
    while (i<len(paras)):
        p = paras[i]
        if contains_sliding_window(p.text.lower(), target, 2 * len(keyword)):
            print 'test3'
            index.append(i)
        i = i+1
    return index

In [65]:
'''
Author: Eden

Find the keywords by sliding windows so that even the order of keywords change, we can still fetch the results

-text = all paragraphs
-keywords = search words
-win_len = the length of sliding window

return True if the keyword is in text within a sliding window by random orders.
'''
def contains_sliding_window(text, keyword, win_len):
    target_words = re.split('[\s]', keyword)
    probe = target_words[0]
#     print probe
#     print text
    
    flag = False
    i = 0
    while (True):
        text = text[i:]
        if probe in text:
            i = text.index(probe)
            l_pointer = max(0,i - win_len)
            r_pointer = min(len(text),i + win_len)
            window_str = text[l_pointer:r_pointer]
            flag = True
            for target_word in target_words:
                if target_word not in window_str:
                    flag = False
            if flag:
                return True
        else:
            break

In [66]:
'''
Testing Function
'''
text = 'aaChange of Control means an event or series of events by which:'
if contains_sliding_window(text.lower(), 'change control', 2 * len(keyword)):
    print text + '1'

aaChange of Control means an event or series of events by which:1


### upper_camel_case(para, keywords) Example:
Capitalized Lease Obligations means, with respect to any Person, all outstanding obligations of such Person in respect of Capital Leases, taken at the capitalized amount thereof accounted for as indebtedness in accordance with GAAP.

Change of Control means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

Code shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [67]:
'''
Author: Zoe

Still Testing
'''
def upper_camel_case(para, stop_set):
    words_list = re.split(" ", para.text)
    new_para = remove_stopwords(para.text, stop_set)
    new_para_list = re.split(" ", new_para)
    new_words_list = re.findall(r'([A-Z][a-z0-9\']+)[^A-Z]?\s', new_para)
    last_word = ""
    for i in range(len(new_words_list)):
        if new_words_list[i] == new_para_list[i]:
            last_word = new_words_list[i]
        else:
            break
    result = []
    for j in range(len(words_list)):
        if last_word == words_list[j]:
            result.append(last_word)
            break
        else:
            result.append(words_list[j])
    if len(result) > 1:
        return True
    else:
        return False

### is_upper_camel_case() Example:
Change of Control means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.


In [68]:
'''
Author: Zoe

Still Testing
'''
def is_upper_camel_case(paragraph, keyword, stop_set):
    words_list = re.compile('\w+').findall(paragraph.text)
    new_paragraph = remove_stopwords(paragraph.text, stop_set)
    new_paragraph_list = re.compile('\w+').findall(new_paragraph)
    new_words_list = re.findall(r'([A-Z][a-z0-9\']+)[^A-Z]?\s', new_paragraph)

    last_word = ""
    for i in range(len(new_words_list)):
        if new_words_list[i] == new_paragraph_list[i]:
            last_word = new_words_list[i]
        else:
            break
    result = []
    for j in range(len(words_list)):
        if last_word == words_list[j]:
            result.append(last_word)
            break
        else:
            result.append(words_list[j])
    results = " ".join(result)
    if keyword.lower() in results.lower():
        return True
    else:
        return False

In [28]:
# s = is_upper_camel_case('''"Change of Control" means Other Connection Taxes that are imposed on or measured by net income (however denominated) or that are franchise Taxes or branch profits Taxes.
# ''', 'change of control')
# print s

In [69]:
'''
Author: Yufei

Traversing through each of the paragraphs and find the paragraph containing the keywords

-paras = all paragraphs
-keywords = search words

return a list of paragraph index 
'''
def search(paras, keywords):
    index = []
    i = 0
    while (i<len(paras)):
        p = paras[i]
        if keywords.lower() in p.text.lower():
            index.append(i)
        i = i+1
    return index

In [70]:
'''
Author: Yufei

Find the pattern of the paragraph containing the keywords by call the is_* function

-paras = all paragraphs
-index = paragraph index
-keyword = search words

return True/False list, 
[bold, italic, underline, double_quotes, single_quotes, upper case, (upper camel case), indent space, list paragraph, normal following list paragraph, bullet list]

'''
def find_patterns(paras, index, keyword, stop_set):
    paragraph = paras[index]
    patterns = []
    patterns.append(is_bold(paragraph, keyword))
    patterns.append(is_italic(paragraph, keyword))
    patterns.append(is_underline(paragraph, keyword))
    patterns.append(is_double_quotes(paragraph, keyword, stop_set))
    patterns.append(is_single_quotes(paragraph, keyword, stop_set))
    patterns.append(is_upper_case(paragraph, keyword))
    patterns.append(is_upper_camel_case(paragraph, keyword, stop_set))
    patterns.append(indent(paragraph))
    patterns.append(isListPara(paragraph))
    patterns.append(isNormalFollowingListPara(paras, index))
    patterns.append(bullet_list(paragraph))
    return patterns

In [71]:
'''
Author: Yufei

Find the pattern of the paragraph containing the keywords by call the is_* function

-paras = all paragraphs
-targetPatterns = the target pattern
-startIndex = paragraph index

return content
'''
def match(paras, targetPatterns, startIndex):
    
    i = startIndex + 1
    match = []
    
    print '\tTarget Patterns: ' + str(parttern_translate(targetPatterns))
    
    
    while (i<len(paras)):
        patterns = []
        p = paras[i]
        

        patterns.append(bold(p))
        patterns.append(italic(p))
        patterns.append(underline(p))
        patterns.append(double_quotes(p))
        patterns.append(single_quotes(p))
        patterns.append(upper_case(p))
        patterns.append(upper_camel_case(p, stop_set))
        patterns.append(indent(p))
        patterns.append(isListPara(p))
        patterns.append(isNormalFollowingListPara(paras, i))
        patterns.append(bullet_list(p))
        
        print '\n\tNext Paragraph:\t{0}\n\n\t\t{1}'.format(str(parttern_translate(patterns)), p.text.encode("utf-8"))
        
        if patterns == targetPatterns:
            print 'Bingo Bingo Bingo Bingo'
            break
        i = i + 1
    for x in range(startIndex, i):
        match.append(paras[x])
        
    return match, i-1

In [72]:
'''
Author: Eden

return the list of patterns' name

-pattern = the pattern list with true/false

return list of patterns name
'''
def parttern_translate(pattern):
    list = []
    if pattern[0]:
        list.append('blod')
    if pattern[1]:
        list.append('italic')
    if pattern[2]:
        list.append('underline')
    if pattern[3]:
        list.append('double_quotes')
    if pattern[4]:
        list.append('single_quotes')
    if pattern[5]:
        list.append('upper_case')
    if pattern[6]:
        list.append('upper_camel_case')
    if pattern[7]:
        list.append(pattern[7])
    if pattern[8]:
        list.append('list_paragraph')
    if pattern[9]:
        list.append('normal_following_list_paragraph')
    if pattern[10]:
        list.append('bullet_list')
    return list
    

In [73]:
# # Abhay
# '''
# Open the output file in append mode and pull related language from the files in the input folder
# '''

# ########################################################################
# out_filepath = 'Output/'
# out_filename = 'Output_search.csv'
# out_path = out_filepath + out_filename
# ########################################################################


# header = ['File Name', 'Keywords', 'Change of Control'] # Column names in output file 

# with open(out_path, 'wb') as f:
#     wr = csv.writer(f)
#     wr.writerow(header)

# with open(out_path, 'ab') as f:
#     # Input folder path
#     directory_path = 'input' 
    
#     # Parse through each file in the input folder path
#     for fileName in os.listdir(directory_path):
#         print fileName
        
#         # Create a Document object of each of the files
#         #document = Document(directory_path + "/" +fileName)
#         document = Document('input/upper_camel_case+underline+after_list_paragraph.docx')
        
        
#         # A list to add all the paragraphs in the document
#         paras = []
        
#         '''
#         The following for loop is used to add all the paragraphs in the document to the paras list object
#         '''
#         for p in document.paragraphs:        
#             paras.append(p)
            
#         paras_number = len(paras) # Find the number of paragraphs in the document
        
#         paras_toPrint = [] # A list to add all the paragraphs relevant to 'Change of Control' language in this case
        
#         keyword = "Change of Control"
#         keywordindex = search(paras, keyword)
        
#         print('keywordIndex:', keywordindex)
        
#         results = []
#         result = ""
#         for index in keywordindex:
#             print 'Target Para', paras[index].text
#             target = find_patterns(paras, index, keyword)
            
#             if index == paras_number - 1:
#                 result = paras[index]
#             else:
#                 result = match(paras, target, index)
            
#             results.append(result)

#         wr = csv.writer(f)
#         wr.writerow([fileName, keyword, results])
        
# print("end")
# f.close()

In [84]:

'''
Main Function.

02/17/2018 Eden
1. Modified the System output to make programmers easier to debug.
2. Modified the output format in .csv files
3. encode all output string to be utf-8
4. Added one function to output the patterns name rather than simple true/false
'''

########################################################################
out_filepath = 'output/'
out_filename = 'Output_search.csv'
out_path = out_filepath + out_filename
########################################################################


header = ['File Name', 'Keywords', 'Content'] # Column names in output file 

with open(out_path, 'wb') as f:
    wr = csv.writer(f)
    wr.writerow(header)
    
with open(out_path, 'ab') as f:
    # Input folder path
    directory_path = 'input/change_of_control+change_in_control' 
    
    stop_set = read_stop_words('res/stop_words.txt')    
    
    # Parse through each file in the input folder path
    print 'Keyword: ' + 'Change of Control' + '\n'
    for fileName in os.listdir(directory_path):
        print '-------------------------------------------------------------------------'
        print 'File name: ' + fileName + '\n'
        
        # Create a Document object of each of the files
        document = Document(directory_path + "/" +fileName)
        # document = Document('/upper_camel_case+underline+after_list_paragraph.docx')
        
        
        # A list to add all the paragraphs in the document
        paras = []
        
        '''
        The following for loop is used to add all the paragraphs in the document to the paras list object
        '''
        for p in document.paragraphs:        
            paras.append(p)
            
        paras_number = len(paras) # Find the number of paragraphs in the document
        
        paras_toPrint = [] # A list to add all the paragraphs relevant to 'Change of Control' language in this case
        
        keyword = "Change of Control"
        keywordindex = search(paras, keyword)
        
#         print '\tkeywordIndex: ' + str(keywordindex)
#         print
        
        results = []
        result = ""
        endindex = -1
        for index in keywordindex:
            if not keyword.lower() in paras[index].text[:100].lower():
                results.append(paras[index])
                endindex = index
            print '\tTarget Paragraph: [{0}]\n\t\t{1}\n'.format(index, paras[index].text.encode("utf-8"))
            target = find_patterns(paras, index, keyword, stop_set)
            
            if index <= endindex:
                continue
            if index == paras_number - 1:
                result = paras[index]
            else:
                result, endindex = match(paras, target, index)
            
            results.append(result)

        wr = csv.writer(f)
        # Created a single list of all instances of 'Change of Control'
        # Ignored unrecognized ascii characters that throw an encoding error. Need improvement
        for result in results:
            if hasattr(result, '__iter__'):
                text =  [paras.text.encode('ascii','ignore') for paras in result]
                res = "\n".join(text)
            else:
                res = result.text.encode('ascii','ignore')
            
            wr.writerow([fileName, keyword, res])
        
        print '\n'
print("end")
f.close()

Keyword: Change of Control

-------------------------------------------------------------------------
File name: 10owzQMAwOayg79reaEvNH.docx

	Target Paragraph: [87]
		3.3           Change of Control. If Executive terminates this Agreement or his employment with the Company for Good Reason or if Executive’s employment with the Company is terminated by the Company for any reason other than for Cause, including non-renewal of this Agreement by the Company, and such termination occurs during a Change of Control Period, the Company shall pay to Executive a change of control indemnity of: (i) the Severance Indemnity as defined in Section 3.2; and (ii) a lump-sum payment equal to one hundred percent (100%) of the higher of: (A) the greater of (x) Executive’s target bonus as in effect for the fiscal year in which the Change of Control occurs or (y) Executive’s target bonus as in effect for the fiscal year in which Executive’s termination of employment occurs; or (B) Executive’s actual bonus f


	Next Paragraph:	['blod', 'italic']

		EXECUTIVE:

	Next Paragraph:	['underline', 'upper_camel_case']

		/s/ Steven L. Hartung

	Next Paragraph:	[]

		Steven L. Hartung

	Next Paragraph:	[]

		
	Target Paragraph: [33]
		6. Limit on Severance Payment for Change of Control.

	Target Paragraph: [36]
		a. Executive agrees that for the term of this Agreement and for a period of two (2) years after this Agreement is terminated pursuant to Section 5(a) or (b) (with the understanding that the two (2) year period will be shortened to one (1) year upon the completion of a transaction constituting a Change of Control, as defined in Section 5(f)(i)), Executive will not directly or indirectly be employed by, own, manage, operate, support, join, or benefit in any way from any business activity that is competitive with Employer’s business or reasonably anticipated business of which Executive has knowledge. For purposes of the foregoing, Executive will be deemed to be connected with such business if 


-------------------------------------------------------------------------
File name: 54iMzNb4LjeC6i98Uewx6h.docx

	Target Paragraph: [25]
		“Change of Control” means (a) a merger or consolidation involving the Issuer or a sale, exchange, conveyance or other disposition of voting securities of the Issuer to a person or “group” (within the meaning of Section 13(d)(3) of the Exchange Act), in a single transaction or series of related transactions, if, as a result of such merger, consolidation, sale, exchange conveyance or other disposition, the stockholders of Issuer immediately prior to such merger, consolidation, exchange, conveyance or other disposition (determined at the time of the first of such series of transactions) beneficially own (within the meaning of Section 13(d)(3) of the Exchange Act) less than a majority of the voting power of the Issuer (or, if applicable, successor to the Issuer or acquiring entity (or parent thereof)) immediately after such merger, consolidation, sale


	Next Paragraph:	['underline', 'upper_camel_case']

		(c) Indebtedness outstanding on the date hereof and listed on Schedule 6.01(b) and any refinancing, extension or replacement thereof;

	Next Paragraph:	['upper_camel_case']

		(d) Indebtedness of Issuer or any of its Restricted Subsidiaries to the Issuer or any other Restricted Subsidiary, so long as to the extent such Indebtedness is owing by Issuer, it is subordinated to the obligations hereunder;

	Next Paragraph:	['underline', 'upper_camel_case']

		(e) Guarantees of other Indebtedness permitted pursuant to this Section 6.01;

	Next Paragraph:	['upper_camel_case']

		(f) Indebtedness in respect of Swap Contracts designed to hedge against the Issuer’s or any Restricted Subsidiary’s exposure to interest rates, foreign exchange rates or commodities pricing risks incurred in the ordinary course of business and not for speculative purposes;

	Next Paragraph:	['underline', 'upper_camel_case']

		(g) Indebtedness to current or former 


-------------------------------------------------------------------------
File name: 7hAp6m5JRwE6E6UZKnKHAK.docx

	Target Paragraph: [53]
		(iii) If the Executive’s employment is terminated (A) in connection with a Change of Control, as defined below, (B) by the Company without Cause or (C) upon death or Disability, as defined below, all unvested Awards shall immediately vest and become exercisable effective the date of termination of employment, and, to the extent unexercised, shall expire one year after any such event.

	Target Patterns: ['upper_camel_case', 36.0, 'bullet_list']

	Next Paragraph:	[36.0]

		

	Next Paragraph:	['underline', 'upper_camel_case', 36.0]

		(g) Payment. The full consideration for any shares purchased by the Executive upon exercise of the Option shall be paid in cash.

	Next Paragraph:	[36.0]

		 

	Next Paragraph:	['underline', 'upper_camel_case', 36.0, 'bullet_list']

		10. Other Benefits.

	Next Paragraph:	[36.0]

		

	Next Paragraph:	['underline', 'uppe


	Next Paragraph:	[]

		 

	Next Paragraph:	[]

		If to the Company:

	Next Paragraph:	[]

		

	Next Paragraph:	[]

		Ethanex Energy, Inc.

	Next Paragraph:	['upper_camel_case']

		14500 Parallel Road,

	Next Paragraph:	[]

		Suite A

	Next Paragraph:	['upper_camel_case']

		Basehor, Kansas

	Next Paragraph:	['upper_camel_case']

		Attn: Robert C. Walther, Executive Chairman

	Next Paragraph:	['upper_camel_case']

		Facsimile: (913) 721-5801

	Next Paragraph:	['underline', 'bullet_list']

		r.walther@ethanexenergy.com

	Next Paragraph:	[]

		 

	Next Paragraph:	[]

		If to the Executive:

	Next Paragraph:	[]

		

	Next Paragraph:	[36.0]

		Alan H. Belcher

	Next Paragraph:	['upper_camel_case', 36.0]

		14500 Parallel Road,

	Next Paragraph:	[36.0]

		Suite A

	Next Paragraph:	['upper_camel_case', 36.0]

		Basehor, KS 66007

	Next Paragraph:	['underline', 36.0, 'bullet_list']

		a.belcher@ethanexenergy.com

	Next Paragraph:	['upper_camel_case', 36.0]

		Facsimile: (913) 721-5801

	Next 

In [ ]:
os.getcwd()